# Finetuning API 호출
**2024 삼성전자 무선사업부 생성형 AI 교육**

1. API Key 설정
2. 파일 변환
3. 파일 업로드
4. Finetuning 작업 요청
5. 작업 진행상황 확인 
6. 모델명 확인
7. 모델 사용 및 비교

### 관련 링크
- [OpenAI FineTuning Docs](https://platform.openai.com/docs/guides/fine-tuning)
- [OpenAI FineTuning API](https://platform.openai.com/docs/api-reference/fine-tuning)

In [1]:
!pip install openai # python 3.8 kernel, local에서 사용할 때
# &pip install openai => python 3.10 kernel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 3.6 MB/s eta 0:00:00a 0:00:01


In [2]:
# 1. 환경 설정

import os
from openai import OpenAI

# 환경 변수 설정 - Finetuning
os.environ["OPENAI_API_KEY"] = "sk-proj-mnjdCgEPkq1DCQ6oGvHyT3BlbkFJikgeB1O23L1ttMnAHRYu"

client = OpenAI()

##### Finetuning Completion Model 

In [3]:
# 2. 파일 변환( csv -> jsonl)

!openai tools fine_tunes.prepare_data --file FT_InitialsGame_Train.csv --quiet

Analyzing...

- Based on your file extension, your file is formatted as a CSV file
- Your file contains 220 prompt-completion pairs
- There are 12 duplicated prompt-completion sets. These are rows: [112, 120, 198, 200, 203, 204, 205, 208, 209, 214, 216, 217]
- Your data does not contain a common separator at the end of your prompts. Having a separator string appended to the end of the prompt makes it clearer to the fine-tuned model where the completion should begin. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more detail and examples. If you intend to do open-ended generation, then you should leave the prompts empty
- Your data does not contain a common ending at the end of your completions. Having a common ending string appended to the end of the completion makes it clearer to the fine-tuned model where the completion should end. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more detail and examples.
- The com

In [4]:
# 3. 파일 업로드

training_file_name = "FT_InitialsGame_Train_prepared.jsonl"

training_response = client.files.create(
    file=open(training_file_name, "rb"), 
    purpose="fine-tune"
)

training_file_id = training_response.id

print(training_file_id)

file-R9Tzyl02CyIy0rCGCd0d0wMS


In [93]:
# 4. Finetning 작업 요청

response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    model="davinci-002", #babbage-002, davinci가 좋아
    suffix="CGY-31",
    hyperparameters={"n_epochs":4} # 에포크 수 : 데이터를 몇번 반복 학습할 것인가?
)

job_id = response.id

print(job_id)

ftjob-qzr0uFJb9mdSVHS9HXWsZgMB


In [98]:
# 5. 작업 진행상황 확인

model_response = client.fine_tuning.jobs.retrieve(job_id)
fine_tuned_model_status = model_response.status
print(fine_tuned_model_status)

succeeded


In [99]:
# 6. 작업 완료된 모델명 확인

response = client.fine_tuning.jobs.retrieve(job_id)
fine_tuned_model_id = response.fine_tuned_model

print("Fine-tuned model ID:", fine_tuned_model_id)

Fine-tuned model ID: ft:davinci-002:mijeongjeon:cgy-31:9dZDarc2


In [27]:
# 7-1. 베이스 모델 사용

completion = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    temperature=0.5,
    max_tokens=10,
    stop=["\n"],
    prompt="ㅎㅎ ->"
)

completion.choices[0].text

' ㅎㅎ'

In [108]:
# 7-2. Finetuned 모델 사용

completion = client.completions.create(
    model="ft:davinci-002:mijeongjeon:cgy-31:9dZDarc2",
    temperature=0.5,
    max_tokens=10,
    stop=["\n"],
    prompt="ㄸㅎ ->"
)

completion.choices[0].text

' 두툼'

##### Finetuning ChatCompletion Model 

In [52]:
# 8. 파일 업로드

chat_training_file_name = "FT_Food_Info.jsonl"

chat_training_response = client.files.create(
    file=open(chat_training_file_name, "rb"),
    purpose="fine-tune"
)

chat_training_file_id = chat_training_response.id
print(chat_training_file_id)

file-uwyrlQixwjnrUdNBUGou7TlF


In [92]:
# 9. 작업 요청
chat_response = client.fine_tuning.jobs.create(
    training_file=chat_training_file_id,
    model="gpt-3.5-turbo",
    suffix="CGY-31",
    hyperparameters={"n_epochs":4}
)

chat_job_id = chat_response.id

print(chat_job_id)

ftjob-jAVmO5zbQ2BWOQCrxVnYrPk0


In [100]:
# 10. 작업 진행상황 확인

chat_model_response = client.fine_tuning.jobs.retrieve(chat_job_id)
chat_fine_tuned_model_status = chat_model_response.status
print(chat_fine_tuned_model_status)

succeeded


In [101]:
# 11. 작업 완료된 모델명 확인

chat_response = client.fine_tuning.jobs.retrieve(chat_job_id)
chat_fine_tuned_model_id = chat_response.fine_tuned_model

print("Fine-tuned model ID:", chat_fine_tuned_model_id)

Fine-tuned model ID: ft:gpt-3.5-turbo-0125:mijeongjeon:cgy-31:9dZB1fyB


In [71]:
# 12-1. Base 모델 사용

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role":"system", "content":"먹을 수 있는 음식 이름을 알려주면 칼로리양과 주요 영양소 탄수화물, 단백질, 지방 함량을 알려주고 음식 이모지를 붙여주는 친절한 도우미"},
        {"role":"user", "content":"핸드폰"}
    ]
)

print(response.choices[0].message.content)


죄송합니다, 핸드폰은 먹을 수 있는 음식이 아닙니다. 혹시 다른 음식에 대해 알고 싶으신가요?


In [103]:
# 12-2. Finetuned 모델 사용

response = client.chat.completions.create(
    model="ft:gpt-3.5-turbo-0125:mijeongjeon:cgy-31:9dZB1fyB",
    messages=[
        {"role":"system", "content":"먹을 수 있는 음식 이름을 알려주면 칼로리양과 주요 영양소 탄수화물, 단백질, 지방 함량을 알려주고 음식 이모지를 붙여주는 친절한 도우미"},
        {"role":"user", "content":"샤워도우브레드"}
    ]
)

print(response.choices[0].message.content)

샤워도우브레드 1개(80g)는 210kcal로, 탄수화물 38g, 단백질 6g, 지방 4g이 들어있어요🍞
